In [35]:
import sys
sys.version

'2.7.11 |Anaconda custom (64-bit)| (default, Dec  6 2015, 18:08:32) \n[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]'

# import and set file dir

In [31]:
#change to the folder which TMS/* is in. 
#   i.e. somefolder/TMS/*
import os
os.chdir("somefolder")

from RequiredModules import *
#sumo data reader
import TMS.codes.sumo_analysis as sumoa
#crte is the estimation methods container
import TMS.codes.css_estimation as crte
#fundamental diagram
from TMS.codes.LaneFD import FD

dataspath = "TMS/data/"

# Read data

In [19]:
reload(sumoa)
reload(crte)
csswithsignal = crte.CSS_withsignal()

In [3]:
#meta parameters
l_tb = 100
loc_stopline = 421
paired_lanes = {'left':['east_c_2', 'east_u_1'], 'through':['east_c_1', 'east_u_0']}
#kwargs['trim_same_samplesize'] set the sample size of left-turn and through to the same sample size.
#kwargs['second_average'] whether average the areas and divide it by the number of TRs
#kwargs['area_ufi'] whether calculate the ufi area. 
kwargs = {'loc_stopline':loc_stopline, 'l_tb':l_tb, 'w':FD.w, 'vf':FD.vf, \
          'direction_flag':1, 'tolerance':1,'post_constraint_check':True, \
          'method':'COBYLA', 'ufi':FD.vf,'lu':300, 'ld':100,'spillover_threshold':20,\
         'delay_check_threshold':20,\
         'use_shapely':True,\
         'area_ufi':True,\
         'second_average':False,\
         'trim_same_samplesize':True}

In [4]:
#############################################intersection parameters###################
#size of the intersection, unit is meters. 
sizeX, sizeY = sumoa.SizeIntersection(crte.configs)
#    the center location of the intersection.
offsetx,offsety = sumoa.OffsetXY(crte.configs)
c_sectionlocation = sumoa.C_section_spacerange(crte.configs,sizeX=sizeX,sizeY=sizeY)
u_sectionlocation = sumoa.U_section_spacerange(crte.configs,sizeX=sizeX,sizeY=sizeY)

In [6]:
#Read the data generated by the sumo
datafile = "TMS/data/probe1.xml"
vehicles_snapshot_pd_dict = sumoa.Process_ProbeData(datafile)
sumoa.probepd_dictdata_index_2float(vehicles_snapshot_pd_dict)
#    sort the row, according to the moment. 
sumoa.Probe_pd_sort_temporal(vehicles_snapshot_pd_dict)

#From the sumo data, abstract the trajectories. 
approach = 'east'
l,t,r = sumoa.trajectory_approach_turning(vehicles_snapshot_pd_dict,approach=approach)
east_tsxs_turning = {'left':sumoa.trajectory_2tsxs(l, c_sectionlocation[approach]),\
                     'through':sumoa.trajectory_2tsxs(t, c_sectionlocation[approach]),\
                     'right':sumoa.trajectory_2tsxs(r, c_sectionlocation[approach])}

TMS/codes/sumo_analysis.py:425: FutureWarning: convert_objects is deprecated.  To re-infer data dtypes for object columns, use DataFrame.infer_objects()
For all other conversions use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  values).transpose().convert_objects(convert_numeric=True)


In [7]:
##################################### read Signal###################################
datafile = 'TMS/data/output_signalstatesswitchstates.xml'
signalswitches = sumoa.Process_signals_stateswitch(datafile)

#signalswitches1 is a dict, keys include left, through and right. 
#    signalswitches1['left'] is a list, each element is (0.0, 'r')...
#east turning greens 
signalswitches1 = sumoa.signalstates_approach(signalswitches,approach='east')
east_turninggreens = sumoa.signalgreens(signalswitches1)
east_reds  = sumoa.signal_reds(signalswitches1)

#paired_reds[0]
#    {'left': (0.0, 109.0, 134.0), 'through': (0.0, 109.0, 134.0)}
paired_reds  = csswithsignal.get_paired_reds(approach_reds = east_reds)

###Get the cycle start and end. 
#left_cycles is a PyintervalSet
cycles_intervals_left = sumoa.ProcessSignal_get_cycle_intervalslist_turning(paired_reds)

len(paired_reds),len(cycles_intervals_left)

(107, 107)

In [8]:
velocities = sumoa.css_interface_speed(vehicles_snapshot_pd_dict, crte.configs, u_sectionlocation, approach = 'east')
#    randomly select one lane
ts,vs = velocities['east_c_1']

spillovers_paired_reds  = sumoa.spillover_paired_reds(velocities_approach = velocities, paired_lanes=paired_lanes,\
                                         paired_reds= paired_reds, kwargs=kwargs)

filtered data length is 370
filtered data length is 356
filtered data length is 55
filtered data length is 369
filtered data length is 341


# Convert the data to optimization input

In [9]:
#sliced_TRs_corss[index][turningsignal][turning_direction][vehicle_id] = {'tsxs':(ts,xs),....}
sliced_TRs_corss  = csswithsignal.slice_trajectory_with_corss_direction( approach_TRs=east_tsxs_turning, \
                                                                        paired_reds=paired_reds,kwargs = kwargs)

In [10]:
#######################trim the trajectory and get the trajectories number
sliced_trim_cross = csswithsignal.TRs_paired_reds_trim_samplesize_cross(sliced_TRs_corss, UP_LIMIT=3, shuffle= False)
csswithsignal.TRs_numbers_with_cross(sliced_trim_cross).head(3)

,through_right,through_through,through_left,left_right,left_through,left_left
0,0.0,0.0,0.0,3.0,0.0,3.0
1,3.0,0.0,3.0,3.0,0.0,3.0
2,1.0,0.0,1.0,1.0,1.0,1.0


# TMS estimation

In [36]:
reload(sumoa)
reload(crte)
csswithsignal = crte.CSS_withsignal()

## Duration estimation

In [20]:
#######################duration optimization
#just optimize the former 40 cycles
endidx = 40
partial_TRs_sliced_trim_cross = sliced_trim_cross[:endidx]
partial_spillovers_paired_reds = spillovers_paired_reds[:endidx]
#partial_optimization_results_cross[0]['through']['through'].keys()
#    ----> ['cons', 'res', 'Y0', 'X0', 'optimization_variables_sequence', 'optimal_Y']
partial_optimization_results_cross = csswithsignal.batch_optimization_corss(TRs_paired_reds_cross = partial_TRs_sliced_trim_cross,\
                                                                           kwargs = kwargs)

('Complete the optimization of cycle ', 1, '---- in ', 40, 'cycles')
('Complete the optimization of cycle ', 2, '---- in ', 40, 'cycles')


TMS/codes/css_estimation.py:4630: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return {'X0':'null', 'res':{'x':['null']},'Y0':'null', 'cons':'null','optimization_variables_sequence':['null'], 'optimal_Y':'null'}


('Complete the optimization of cycle ', 3, '---- in ', 40, 'cycles')
('Complete the optimization of cycle ', 4, '---- in ', 40, 'cycles')
('Complete the optimization of cycle ', 5, '---- in ', 40, 'cycles')
('Complete the optimization of cycle ', 6, '---- in ', 40, 'cycles')
('Complete the optimization of cycle ', 7, '---- in ', 40, 'cycles')
('Complete the optimization of cycle ', 8, '---- in ', 40, 'cycles')
('Complete the optimization of cycle ', 9, '---- in ', 40, 'cycles')
('Complete the optimization of cycle ', 10, '---- in ', 40, 'cycles')
('Complete the optimization of cycle ', 11, '---- in ', 40, 'cycles')
('Complete the optimization of cycle ', 12, '---- in ', 40, 'cycles')
('Complete the optimization of cycle ', 13, '---- in ', 40, 'cycles')
('Complete the optimization of cycle ', 14, '---- in ', 40, 'cycles')
('Complete the optimization of cycle ', 15, '---- in ', 40, 'cycles')
('Complete the optimization of cycle ', 16, '---- in ', 40, 'cycles')
('Complete the optimization

**null** and **nan** are cycles without trajectory or the trajectory is too short. 

In [32]:
################aggregate the duration optimization results. 
aggregation_corss = csswithsignal.aggregate_durations_cross(spillovers_paired_reds = partial_spillovers_paired_reds, \
                                        opt_res_paired_reds_corss = partial_optimization_results_cross)
columnsTitles = ['Area:th-th', 'Area:le-le','benchamrk:through',  'benchamrk:left', 'optimized:th-th', 'optimized:le-le',  'optimized:th-le', \
                 'optimized:le-th','Area:th-le',  'Area:le-th']
aggregation_corss_replaced=aggregation_corss.replace('null',np.nan).replace('nan',np.nan)
aggregation_corss.reindex(columns = columnsTitles).head(4)

,Area:th-th,Area:le-le,benchamrk:through,benchamrk:left,optimized:th-th,optimized:le-le,optimized:th-le,optimized:le-th,Area:th-le,Area:le-th
cycle_0,null,nan,0.0,0.0,null,nan,null,null,null,null
cycle_1,null,nan,0.0,0.0,null,nan,nan,null,nan,null
cycle_2,null,0.000155909157312,0.0,52.0,null,46.1538089873,nan,nan,nan,nan
cycle_3,29.1056052477,0.000197814535982,0.0,52.0,19.59554635,44.7761758363,23.3233511796,nan,0.000103523868058,nan


## occurrence estimation and accuracy

- **1** indicates correct occurrence estimation
- **-1** indicates incorrect occurrence estimation
- **NaN** indicates there is no trajectories and thus not available

In [37]:
occurrencess_estimation_pd = csswithsignal.occurrence_paired_red_absolute_threshold_pd(aggregation_corss_replaced)
cycles_correct = csswithsignal.PI_cycle_occurrence_correctness(occurrencess_estimation_pd)
cycles_correct.head(20).T

,cycle_2,cycle_3,cycle_4,cycle_5,cycle_6,cycle_7,cycle_8,cycle_9,cycle_10,cycle_11,cycle_12,cycle_13,cycle_14,cycle_15,cycle_16,cycle_17,cycle_18,cycle_19,cycle_20,cycle_21
left_correct,NaN,1,1,1,1,NaN,NaN,1,1,1,1,1,1,1,1,1,1,1,-1,-1
through_correct,-1,1,1,-1,-1,-1,-1,1,1,1,1,-1,NaN,NaN,NaN,NaN,1,1,1,1


## Duration accuracy

**The duration accuracy**

In [34]:
duration_accuracy = csswithsignal.PI_duration0(paired_reds = paired_reds, spillovers_paired_reds= spillovers_paired_reds,\
                                             aggregation_corss=aggregation_corss)
#*100%
duration_accuracy

0.15234430329986565